In [1]:
from pulp import *
import pandas as pd
import numpy as np

In [2]:
prob = LpProblem("hornby",LpMinimize)

In [3]:
df_cost = pd.read_excel("Exhibits.xlsx",sheet_name="cost")
df_demand = pd.read_excel("Exhibits.xlsx",sheet_name="demand")
df_dis_cost = pd.read_excel("Exhibits.xlsx",sheet_name="dis_cost")
df_dis_cost.set_index("Unnamed: 0",inplace=True)

In [4]:
from_cities = df_cost.Location.tolist()#8
to_cities = df_demand.Location.tolist()#22

fixed_cost = df_cost["Fixed cost"].tolist()
fixed_cost = dict(zip(from_cities,fixed_cost))

var_cost = df_cost["Variable cost"].tolist()
var_cost = dict(zip(from_cities,var_cost))

Min_A = df_cost["Min_A"].tolist()
Min_A = dict(zip(from_cities,Min_A))

Max_A = df_cost["Max_A"].tolist()
Max_A = dict(zip(from_cities,Max_A))

dis_cost = makeDict([from_cities,to_cities],\
                    df_dis_cost.values.transpose().tolist())

In [5]:
demand = df_demand.Volume.tolist()
demand = dict(zip(to_cities,demand))
# Decision Variable
open_or_not = LpVariable.dicts("x",from_cities,lowBound=0,cat="Binary")
product = LpVariable.dicts("p",(from_cities,to_cities), lowBound =0, cat="Continuous")
open_extra = LpVariable.dicts("open extra",from_cities,lowBound=0,cat="Binary")
low_cost = LpVariable.dicts("x",from_cities,lowBound=0,cat="Continuous")
high_cost = LpVariable.dicts("hc",from_cities,lowBound=Min_A,cat="Continuous")

In [6]:
prob += lpSum([dis_cost[i][j]*product[i][j] for i in from_cities for j in to_cities])+\
      lpSum([fixed_cost[i]*open_or_not[i] for i in from_cities])+\
      lpSum([high_cost[i]*var_cost[i] for j in to_cities for i in from_cities])

In [7]:
#Constraints
# products should be smaller than Max_A
for i in from_cities:
    prob += lpSum([product[i][j] for j in to_cities]) <= open_or_not[i]*Max_A[i]

#Low cost only happens when product[i] <= Min_A[i]
for i in from_cities:
    prob += low_cost[i]<=Min_A[i]
    
for i in from_cities:
    prob += low_cost[i] + high_cost[i] == lpSum([product[i][j] for j in to_cities])

for i in from_cities:
    prob += high_cost * open_extra[i] > 0
    
#Products should satisfy the demand    
for j in to_cities:
    prob += lpSum([product[i][j] for i in from_cities])>=demand[j]

TypeError: Non-constant expressions cannot be multiplied

In [ ]:
prob.writeLP("HornBy.lp")
prob.solve()
print("Status:",LpStatus[prob.status])
print(value(prob.objective))

In [ ]:
output=[]
for i in from_cities:# 8 rows
    var_output=[]
    for j in to_cities: # 22 columns
        var_output.append(product[i][j].varValue)
    var_output.append(open_or_not[i].varValue)
    output.append(var_output)
to_cities.append('Rent')
output_df = pd.DataFrame(output,index=from_cities, columns=to_cities)
output_df